# Remote Timing Attacks are Practical
TODO: Intro

## Theory

## Attack Setup

## Exploratory Analysis
The aim of this notebook is to sample the server response times for a bunch of `g` values, and find out if the distribution is meaningfully different between them.

In [ ]:
import tls
import attack

In [ ]:
import scipy
import seaborn
import sympy
import pandas

In [ ]:
raw_samples = attack.bruteforce_most_significant_bits()

In [ ]:
samples = pandas.DataFrame.from_records(raw_samples, columns=['point', 'time'])

Since we know the factors of `N` (`q` and `p` with `q<p`) we can consider each point relative in size to these factors. When plotting, we are looking for peaks and troughs near whole multples of `p` and `q`.

In [ ]:
p = sympy.Integer(11693128827090800677443535237632476895247105886644942164014088484470194179491435241190389270827811769965853291192455791684691555403909415703633832493911789)
q = sympy.Integer(11353860437120204348539420361367294927683441924641720282978666316144621735920188475867378638813811676070362003602263559496393696538309271007870774914687283)
N= sympy.Integer(132762152776056020551326919245624484615462467876809681535549565118332290525598572815747323476102181376625279228965473106140757139049665124368186142774966643990206422037551427526013151129106319233128471783533673959766053786798472937188481868923726256436384468384858420931063093337134977283618537887974322079287)

In [ ]:
samples['point'] = samples['point'].apply(sympy.Integer)

samples['point_relative_to_p'] = samples['point'].apply(lambda g: round(float(g/p), 5))
samples['point_relative_to_q'] = samples['point'].apply(lambda g: round(float(g/q), 5))
samples['point_relative_to_N'] = samples['point'].apply(lambda g: round(float(g/N), 5))

Plot the distribution of timings for each point sampled in the above dataframe.

In [ ]:
g = seaborn.FacetGrid(samples, col="g_relative_to_q", col_wrap=4,   height=2.8, aspect=12/8)
g.map(seaborn.distplot, "time", fit=scipy.stats.norm, kde=None)

Maybe we want to clean this dataset up and remove any outliers. These could be caused by hardware interrupts, funny process scheduling etc.

In [ ]:
mean = samples.groupby(by='point').mean()
std_dev = samples.groupby(by='point').std()

get_mean = {point: mean[0] for point, mean in mean.iterrows()}
get_std_dev = {point: std_dev[0] for point, std_dev in std_dev.iterrows()}

points = list(mean.keys())

group_max = {point: get_mean[point] + 3*get_std_dev[point] for point in points}
group_min = {point: get_mean[point] - 3*get_std_dev[point] for point in points}

def is_outlier(row):
    return row['time'] < group_min[row['point']] or row['time'] > group_max[row['point']]

clean_samples = samples[samples.apply(is_outlier, axis='columns', result_type='reduce') == False]

len(clean_samples)/float(len(samples))

Now we can look at their clean distributions:

In [ ]:
g = seaborn.FacetGrid(clean_samples, col="g_relative_to_q", col_wrap=4,   height=2.8, aspect=12/8)
g.map(seaborn.distplot, "time", fit=scipy.stats.norm, kde=None)

Next, we can consider how the mean response time varies as we vary `g`:

In [ ]:
g = seaborn.relplot(data=clean_times, x='g_relative_to_q', y='time', kind='line',  height=8, aspect=16/8)
g.set(xlim=(0.5, 2.5))
g

Ideally, we want the above graph relation to resemble a noisier Figure 1 from the original paper [1]. It doesn't seem to, or atleast, it is perhaps _too_ noisy...

### Replicating Figure 2

### Replicating Figure 3

## ...

## References
  - [1] https://crypto.stanford.edu/~dabo/papers/ssl-timing.pdf